In [31]:
def help():
    print("P_name is the name of the project e.g AM_BLOTZ_____Cu")
    print("NetID is the API ID of the user's Materials Project")
    print("Element is the dictionary of elements e.g {'CU':1}")
    print("Number of electrons in the valance band e.g. -4 for conventional copper")
    print("SCF, NSCF, AMSET_VASP are the mesh sizes of the regarding VASP caluculations")
    print("temp is the list of temperatures for the calculation")
    print("AMSET_interpolation is the interpolation precision for the AMSET calculation (20-50)")
    print("Run relaxation\n\
    Run SCF\n\
    Run NSCF\n\
    Run BOLTZTRAP\n\
    Run Deform\n\
    Run Elastic\n\
    Run Dielectric\n\
    Run AMSET")

class electrical:
    def __init__(self, api_code,P_name, NetID, elements, SCF_mesh, NSCF_mesh, temp, AMSET_VASP_mesh, AMSET_interpolation):
        self.api_code = api_code
        self.P_name = P_name
        self.pr = Project(P_name)
        self.NetID = NetID
        self.elements = elements
        self.num_valance = num_valance
        self.SCF_mesh = SCF_mesh
        self.NSCF_mesh = NSCF_mesh
        self.temp = temp
        self.AMSET_VASP_mesh = AMSET_VASP_mesh
        self.AMSET_interpolation = AMSET_interpolation
         
    def structure_relaxation(self):
        pr = Project(P_name)
        elements_structures_lst = []
        for element in elements.keys():
            results = MPRester(api_code).query({"formation_energy_per_atom":0,"pretty_formula" : element} , ['cif','energy_per_atom'])
            parser = CifParser.from_string(results[0]['cif'])
            structure = parser.get_structures()[0]
            SGA = SpacegroupAnalyzer(structure)
            conventional_structure = SGA.get_conventional_standard_structure()
            elements_structures_lst.append([conventional_structure, SGA.get_conventional_to_primitive_transformation_matrix()])
        elements_structures_dict = dict(zip(elements, elements_structures_lst))
    
        for name, structure in elements_structures_dict.items():
            job_structure_relaxation = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='structure_relaxation_' + name)
            job_structure_relaxation.structure = self.pr.create.structure.from_pymatgen(structure[0])
            job_structure_relaxation.calc_minimize(pressure=0)
            job_structure_relaxation.input.incar["NCORE"] = 4
            job_structure_relaxation.input.incar["ISMEAR"] = 1
            job_structure_relaxation.input.incar["EDIFF"] = 1e-7
            job_structure_relaxation.server.queue = job_structure_relaxation.server.list_queues()[0] 
            job_structure_relaxation.run()
            atoms = job_structure_relaxation.get_structure()
            return atoms
    
    def Run_SCF(self):
        job_SCF = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='SCF_' + next(iter(elements)))
        job_SCF.structure = self.structure_relaxation()
        job_SCF.input.incar["NCORE"] = 1
        job_SCF.input.incar["ISTART"] = 0
        job_SCF.input.incar["ICHARG"] = 2
        job_SCF.input.incar["ISMEAR"] = 0
        job_SCF.input.incar["NSW"] = 0
        job_SCF.input.incar["EDIFF"] = 1e-7
        job_SCF.input.incar["SIGMA"] = 0.1
        job_SCF.input.incar["ISYM"] = 1
        job_SCF.input.incar["LWAVE"] = True
        job_SCF.server.queue = job_SCF.server.list_queues()[0]
        job_SCF.set_kpoints(mesh=SCF_mesh)
        return job_SCF.run()        
            
    def Run_NSCF(self):
        job_NSCF = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='SCF_' + next(iter(elements))).restart_from_charge_density(job_name="NSCF_" + next(iter(elements)), job_type="Vasp")
        job_NSCF.input.incar["NCORE"] = 4
        job_NSCF.input.incar["ISTART"] = 1
        job_NSCF.input.incar["ICHARG"] = 11
        job_NSCF.input.incar["ISMEAR"] = 0
        #==================================
        job_NSCF.input.incar["LORBIT"] = 11
        job_NSCF.input.incar["NEDOS"] = 1000
        #==================================
        job_NSCF.input.incar["IBRION"] = -1
        job_NSCF.input.incar["NSW"] = 0
        job_NSCF.input.incar["EDIFF"] = 1e-7
        job_NSCF.input.incar["ISYM"] = 1
        job_NSCF.server.queue = job_NSCF.server.list_queues()[0]
        job_NSCF.set_kpoints(mesh=NSCF_mesh)
        return job_NSCF.run()        
    
    def BOLTZTRAP(self):
        for sim_type in ['SCF', 'NSCF']:
            tar_file = '/scratch/user/%s/%s/%s_%s_hdf5/%s_%s/' %(NetID, P_name, sim_type,next(iter(elements)), sim_type,next(iter(elements)))
            file = tarfile.open(tar_file+ '/%s_%s.tar.bz2' %(sim_type, next(iter(elements)) ), "r:bz2")
            file_obj = file.extractfile('vasprun.xml')
            f1 = open(tar_file + 'vasprun.xml','wb')
            f1.write( file_obj.read() )
            f1.close()
            file_obj = file.extractfile('POSCAR')
            f2 = open(tar_file + 'POSCAR','wb')
            f2.write( file_obj.read() )
            f2.close()
            file_obj = file.extractfile('OUTCAR')
            f2 = open(tar_file + 'OUTCAR','wb')
            f2.write( file_obj.read() )
            f2.close()
            file.close()
            
        from BoltzTraP2 import dft as BTP
        from BoltzTraP2 import sphere
        from BoltzTraP2 import fite
        from BoltzTraP2 import bandlib
        from BoltzTraP2 import serialization
        from BoltzTraP2 import units
        from BoltzTraP2 import bandlib as BL
        import matplotlib.pylab as pl

        data = BTP.DFTData('/scratch/user/%s/%s/NSCF_%s_hdf5/NSCF_%s' %(NetID,P_name, next(iter(elements)), next(iter(elements))))
        bt2file = "%s.bt2" %next(iter(elements))

        data.bandana(emin=data.fermi - .2, emax=data.fermi + .2)
        equivalences = sphere.get_equivalences(data.atoms, data.magmom,len(data.kpoints) * 5)
        coeffs = fite.fitde3D(data, equivalences)
        serialization.save_calculation(bt2file, data, equivalences, coeffs,serialization.gen_bt2_metadata(data, data.mommat is not None))
        data, equivalences, coeffs, metadata = serialization.load_calculation(bt2file)
        lattvec = data.get_lattvec()
        eband, vvband, cband = fite.getBTPbands(equivalences, coeffs, lattvec)
        TEMP = np.array(temp) #should be a list [300]
        epsilon, dos, vvdos, cdos = BL.BTPDOS(eband, vvband, npts=4000)

        margin = 9. * units.BOLTZMANN * TEMP.max()
        mur_indices = np.logical_and(epsilon > epsilon.min() + margin,
                                     epsilon < epsilon.max() - margin)
        mur = epsilon[mur_indices]
        N, L0, L1, L2, Lm11 = BL.fermiintegrals(epsilon, dos, vvdos, mur=mur, Tr=TEMP, dosweight=data.dosweight)
        UCvol = data.get_volume()
        sigma, seebeck, kappa, Hall = BL.calc_Onsager_coefficients(L0, L1, L2, mur, TEMP, UCvol)
        SIGMA = [sigma[0][i].trace()/3 for i in range(len(sigma[0]))]
        SEEBECK = [seebeck[0][i].trace()/3 for i in range(len(seebeck[0]))]
        KAPPA = [kappa[0][i].trace()/3 for i in range(len(kappa[0]))]
        return SIGMA, SEEBECK, KAPPA
    
    def Run_Deform(self):
        #====== Setup the Batch file to submit amset wave and amset deform
        path1 = '/scratch/user/%s/%s/SCF_%s_hdf5/SCF_%s' %(NetID, P_name, next(iter(elements)), next(iter(elements)) )
        f = open('%s/Batch.sh' %path1,'w')
        f.write("#!/bin/bash\n\
#SBATCH -J AMSET_Create\n\
#SBATCH -t 2:00:00\n\
#SBATCH -N 1\n\
#SBATCH --ntasks-per-node=2\n\
#SBATCH --mem=32G\n\
#SBATCH -o JobName.o%j\n\
#SBATCH -e JobName.e%j\n\
ml load GCC/10.3.0  OpenMPI/4.1.1\n\
ml load AMSET/0.4.15\n\
amset wave\n\
amset deform create")
        f.close()
        os.chmod('%s/Batch.sh' %path1, 0o0777)
        
        create = subprocess.Popen(['%s/Batch.sh' %path1] \
                          , stdout=subprocess.PIPE, stderr=subprocess.PIPE, \
                         cwd = '%s/' %path1)
        error = create.stderr.read()
        out = create.stdout.read()
        print('\n\n')
        print('wait for 1 minutes')
        time.sleep(60)
        #====== Create deformed structures, run them, and extract the required file for the next section from the zip file
        
        deform = []
        for file in os.listdir(path1):
            if file.startswith("POSCAR-"):
                deform.append( os.path.join(file) )
 
        import ase
        from pyiron import ase_to_pyiron
        for poscar,num in zip(deform, range(len(deform))): 
            structure = ase_to_pyiron( ase.io.read( '%s/%s' %(path1,poscar) ) )
            globals()['deform-%s' %num] = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='deform_' + str(num))
            globals()['deform-%s' %num].structure = structure
            globals()['deform-%s' %num].input.incar["NCORE"] = 1
            globals()['deform-%s' %num].input.incar["ISTART"] = 0
            globals()['deform-%s' %num].input.incar["ICHARG"] = 2
            globals()['deform-%s' %num].input.incar["ISMEAR"] = 0
            globals()['deform-%s' %num].input.incar["NSW"] = 0
            globals()['deform-%s' %num].input.incar["EDIFF"] = 1e-7
            globals()['deform-%s' %num].input.incar["SIGMA"] = 0.1
            globals()['deform-%s' %num].input.incar["ISYM"] = 1
            globals()['deform-%s' %num].input.incar["LWAVE"] = True
            globals()['deform-%s' %num].set_kpoints(mesh=AMSET_VASP_mesh)
            globals()['deform-%s' %num].server.queue = globals()['deform-%s' %num].server.list_queues()[0]
            globals()['deform-%s' %num].run()
        print('wait for a couple of minutes')
        self.pr.wait_for_job(globals()['deform-%s' %num])
        print('Deformed Structure VASP jobs are done.')
        path2 = '/scratch/user/%s/%s' %(NetID, P_name)
        for num in range( len(deform) ):
            tar_file = path2 + '/deform_%s_hdf5/deform_%s/' %(num,num)
            file = tarfile.open(tar_file+ 'deform_%s.tar.bz2' %( num ), "r:bz2")
            file_obj1 = file.extractfile('OUTCAR')
            file_obj2 = file.extractfile('vasprun.xml')
            f1 = open(tar_file + 'OUTCAR','wb')
            f1.write( file_obj1.read() )
            f2 = open(tar_file + 'vasprun.xml','wb')
            f2.write( file_obj2.read() )
            file.close()
            f1.close()
            f2.close()
            
        #====== Set up the Batch file to run the amset deform read to get the deformation potential
        
        deform_result = [path1 if num==0 else\
                         '/scratch/user/%s/%s/deform_%s_hdf5/deform_%s/' %(NetID, P_name,num-1,num-1) \
                         for num in range( len(deform)+1 )]
        deform_result = " ".join(deform_result)
        f = open('%s/Batch_deform.sh' %path1,'w')
        f.write("#!/bin/bash \n\
#SBATCH -J AMSET_deform\n\
#SBATCH -t 2:00:00\n\
#SBATCH -N 1\n\
#SBATCH --ntasks-per-node=2\n\
#SBATCH --mem=32G\n\
#SBATCH -o JobName.o%%j\n\
#SBATCH -e JobName.e%%j\n \
ml load GCC/10.3.0  OpenMPI/4.1.1\n\
ml load AMSET/0.4.15\n\
amset deform read %s" %deform_result)
        f.close()
        os.chmod('%s/Batch_deform.sh' %path1, 0o0777)
        
        deform_job = subprocess.Popen(['%s/Batch_deform.sh' %path1] \
                                  , stdout=subprocess.PIPE, stderr=subprocess.PIPE, \
                                 cwd = '%s/' %path1)
        error = deform_job.stderr.read()
        out = deform_job.stdout.read()

        
    def Run_Elastic(self):
        job_elstic = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='ELASTIC_' + next(iter(elements)), delete_existing_job=True)
        job_elstic.structure = self.structure_relaxation()
        job_elstic.input.incar["NCORE"] = 2
        job_elstic.input.incar["ALGO"] = 'Normal'
        job_elstic.input.incar["ADDGRID"] = True
        job_elstic.input.incar["ISIF"] = 3
        job_elstic.input.incar["IBRION"] = 6
        job_elstic.input.incar["NSW"] = 1
        job_elstic.input.incar["EDIFF"] = 1e-7
        job_elstic.input.incar["EDIFFG"] = -0.01
        job_elstic.input.incar["ISYM"] = 0
        job_elstic.set_kpoints(mesh=AMSET_VASP_mesh)
        job_elstic.server.queue = job_elstic.server.list_queues()[0]
        return job_elstic.run()
        
    def Run_Dielectric(self):
        job_dilectric = self.pr.create_job(job_type=self.pr.job_type.Vasp, job_name='DIELECTRIC_' + next(iter(elements)), delete_existing_job=True)
        job_dilectric.structure = self.structure_relaxation()
        job_dilectric.input.incar["NCORE"] = 2
        job_dilectric.input.incar["ALGO"] = 'Normal'
        job_dilectric.input.incar["ADDGRID"] = True
        job_dilectric.input.incar["ISIF"] = 3
        job_dilectric.input.incar["IBRION"] = 8
        job_dilectric.input.incar["NSW"] = 1
        job_dilectric.input.incar["LEPSILON"] = True
        job_dilectric.input.incar["EDIFF"] = 1e-7
        job_dilectric.input.incar["EDIFFG"] = -0.01
        job_dilectric.input.incar["ISYM"] = 0
        job_dilectric.set_kpoints(mesh=AMSET_VASP_mesh)
        job_dilectric.server.queue = job_dilectric.server.list_queues()[0]
        return job_dilectric.run()
    
    def AMSET(slef):

        #====== Extract the Matrices
        path1 = '/scratch/user/%s/%s/SCF_%s_hdf5/SCF_%s' %(NetID, P_name, next(iter(elements)), next(iter(elements)) )
        f = open( '/scratch/user/%s/%s/DIELECTRIC_%s_hdf5/DIELECTRIC_%s/OUTCAR'%(NetID, P_name, next(iter(elements)), next(iter(elements))) )  
        x = f.read()
        f.close()
        x = x.split('\n')
        index_IONIC = [i for i,a in enumerate(x) if a==' MACROSCOPIC STATIC DIELECTRIC TENSOR IONIC CONTRIBUTION']
        index_HIGHFREQ = [i for i,a in enumerate(x) if a==' MACROSCOPIC STATIC DIELECTRIC TENSOR (including local field effects in DFT)']

        IONIC = x[index_IONIC[0]+2:index_IONIC[0]+5]
        IONIC = [IONIC[i].split() for i in range(len(IONIC))]
        IONIC = [[float(IONIC[j][i]) for i in range(len(IONIC[j]))] for j in range(len(IONIC))]

        HIGHFREQ = x[index_HIGHFREQ[0]+2:index_HIGHFREQ[0]+5]
        HIGHFREQ = [HIGHFREQ[i].split() for i in range(len(HIGHFREQ))]
        HIGHFREQ = [[float(HIGHFREQ[j][i]) for i in range(len(HIGHFREQ[j]))] for j in range(len(HIGHFREQ))]

        IONIC = np.array(IONIC)
        HIGHFREQ = np.array(HIGHFREQ)
        STATIC = IONIC + HIGHFREQ 

        HIGHFREQ = [list(HIGHFREQ[i]) for i in range(len(HIGHFREQ))]
        STATIC = [list(STATIC[i]) for i in range(len(STATIC))]

        STATIC = '\n'.join(['  - '+str(STATIC[i]) for i in range(len(STATIC))])
        HIGHFREQ = '\n'.join(['  - '+str(HIGHFREQ[i]) for i in range(len(HIGHFREQ))])

        f = open( '/scratch/user/%s/%s/ELASTIC_%s_hdf5/ELASTIC_%s/OUTCAR'%(NetID, P_name, next(iter(elements)), next(iter(elements))) )
        x = f.read()
        f.close()
        x = x.split('\n')
        index_ELASTIC = [i for i,a in enumerate(x) if a==' TOTAL ELASTIC MODULI (kBar)']

        ELASTIC = x[index_ELASTIC[0]+3:index_ELASTIC[0]+9]
        ELASTIC = [ELASTIC[i][3:].split() for i in range(len(ELASTIC))]
        ELASTIC = [[float(ELASTIC[j][i]) for i in range(len(ELASTIC[j]))] for j in range(len(ELASTIC))]
        ELASTIC = '\n'.join(['  - '+str(ELASTIC[i]) for i in range(len(ELASTIC))])      
        
        #====== Create the settings.yaml file and a Batch file for submission
        
        f = open( '/scratch/user/%s/%s/SCF_%s_hdf5/SCF_%s/OUTCAR'%(NetID, P_name, next(iter(elements)), next(iter(elements))) )  
        x = f.read()
        f.close()
        x = x.split('\n')
        index_VOL = [i for i,a in enumerate(x) if a==' VOLUME and BASIS-vectors are now :']
        VOL = float( x[index_VOL[0]+3].split()[-1] ) * 1e-24

        num_valance = -4
        doping = num_valance / VOL
        temperature = 300
        interpolation = 20

        f = open('%s/settings.yaml' %path1,'w')
        f.write("# general settings\n\
scattering_type: [ADP]\n\
doping: [%.2e]\n\
temperatures: [%s]\n\n\
interpolation_factor: %s\n\n\n\
deformation_potential: deformation.h5\n\
elastic_constant:\n%s\n\
static_dielectric:\n%s\n\
high_frequency_dielectric:\n%s" %(doping, temperature, interpolation, ELASTIC, STATIC, HIGHFREQ))
        f.close()

        f = open('%s/Batch_amsetrun.sh' %path1,'w')
        f.write("#!/bin/bash \n\
#SBATCH -J AMSET_deform\n\
#SBATCH -t 2:00:00\n\
#SBATCH -N 1\n\
#SBATCH --ntasks-per-node=2\n\
#SBATCH --mem=32G\n\
#SBATCH -o JobName.o%%j\n\
#SBATCH -e JobName.e%%j\n\
ml load GCC/10.3.0  OpenMPI/4.1.1\n\
ml load AMSET/0.4.15\n\
amset run")
        f.close()
        os.chmod('%s/Batch_amsetrun.sh' %path1, 0o0777)

        amset_job = subprocess.Popen(['%s/Batch_amsetrun.sh' %path1] \
                                  , stdout=subprocess.PIPE, stderr=subprocess.PIPE, \
                                 cwd = '%s/' %path1)
        error = amset_job.stderr.read()
        out = amset_job.stdout.read()
        out = out.decode("utf-8").split('\n')
        tau_index = [i for i,a in enumerate(out) if a=='    conc [cm⁻³]    temp [K]       ADP']
        lifetime = lifetime = float( out[tau_index[0]+2].split()[-1] )
        return lifetime


In [32]:
import numpy as np
from pyiron_atomistics import Project
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifParser
import os
import tarfile
import subprocess
import time


api_code = '--------------'
P_name="---------------"
NetID="amirbehbahanian"
elements = {"Cu": 1}
num_valance = -4
SCF_mesh = [4,4,4]
NSCF_mesh = [30,30,30] 
temp = [300]
AMSET_VASP_mesh = [4,4,4]
AMSET_interpolation = 20

Run_test = electrical(api_code, P_name, NetID, elements, SCF_mesh, NSCF_mesh, temp, AMSET_VASP_mesh, AMSET_interpolation)
# Run_test.Run_SCF()
# Run_test.Run_NSCF()
# Run_test.BOLTZTRAP()
# Run_test.Run_Deform()
# Run_test.Run_Elastic()
# Run_test.Run_Dielectric()
Run_test.AMSET()

848000000000.0